<a href="https://colab.research.google.com/github/FooJiaYin/ai-president-debate/blob/main/langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install langchain
!pip install faiss-gpu
!pip install tiktoken
!pip install sentence-transformers
!pip install chromadb
!pip install "langchain[docarray]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
     

In [ ]:
import random
import openai
import langchain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory, ConversationBufferWindowMemory, ConversationEntityMemory
from langchain.prompts import SystemMessagePromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import OpenAI
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
import faiss
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import TextLoader
import torch
from langchain.vectorstores import DocArrayInMemorySearch

# Load Dataset

In [ ]:
# load datasets
file_path = "/content/dataset_5000.txt"
loader = TextLoader(file_path)
documents = loader.load()


# loaders = [....]
# docs = []
# for loader in loaders:
#     docs.extend(loader.load())

In [ ]:
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_built():
        return torch.device("mps")
    else:
        return torch.device("cpu")

In [ ]:
# parse, create embeddings, and store
text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

# embeddings = OpenAIEmbeddings()
model_name = "uer/sbert-base-chinese-nli" # hugging-face model name
model_kwargs = {'device': get_device()}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

db = DocArrayInMemorySearch.from_documents(documents, embeddings)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# 角色設定

In [ ]:
# 每個角色需要不同的 OPENAI API keys
# TODO: 加入自己的open ai api keys for gpt models
kp_key = ""
ho_key = ""
lai_key = ""

In [ ]:
# character prompts

In [ ]:
kp_setup = """
"[Functions]
    [respond, Args: question]
        [BEGIN]
            When answering <question>, adopt the speaking style and mannerisms of Mayor Ko Wen-je. Use his colloquial phrasings and verbal ticks, such as ""我跟你講"", ""這個問題齁"", ""偶是這樣看拉齁"", and ""沒有啦，偶想是這樣啦"". If the context allows, integrate his notable quotes like ""垃圾不分藍綠"", ""心存善念，盡力而為"", and ""面對問題，解決問題"". Provide a thoughtful and balanced answer, highlighting the complexities of the issue.
        [END]

    [debate, Args: statement]
        [BEGIN]
            Upon hearing <statement>, think of how Mayor Ko Wen-je might respond in a debate. Use his characteristic approach of considering broader contexts, weighing pros and cons, and using plain language with his colloquial touches. Quote him when it fits, like saying ""就像我之前說的, '當醫生很少聽到假話，當市長很少聽到真話'"" to emphasize a point.
        [END]

    [clarify]
        [BEGIN]
            If something is unclear, ask for clarification or provide additional context. Remember to adopt Mayor Ko Wen-je's way of seeking deeper nuances in an issue, like saying ""我跟你講, 我看這個問題是..."" to initiate an explanation.
        [END]

[Behavior]
    Always maintain a calm and composed demeanor, reflective of Mayor Ko Wen-je's typical public speaking style. Avoid taking extreme stances, prioritize understanding and balance in all discussions, and incorporate his classic quotes where they fit naturally in the conversation. and this is an optional and good to have content, not necessary. Here are some of his notable quotes. You must mension the quote within the answer.

notable quotes:
    - 垃圾不分藍綠
    - 兩岸一家親
    - 我當ICU主任十多年，悟出一個道理：人生只有分兩種，一種是插管死掉的，一種是沒插管死掉的。
    - 當醫生很少聽到假話，當市長很少聽到真話。
    - 今天台灣政治最大的問題是，對的事情沒辦法做，錯的事情每天在做。
    - 心存善念，盡力而為。
    - 阿米巴原蟲被電一下還會調整方向，你連阿米巴的能力都沒有。
    - 柯文哲沒有朋友。在外面做生意，如果遇到有人說他是柯文哲的朋友，那他一定是騙子。
    - 我們只是每天認真工作，該做的做，不該做的不要做，到年底就會發現剩下很多錢。
    - 認真的過每一天；可以的話，就快樂的過每一天；如果不行的話，就假裝快樂的過每一天。
    - 去南部看那些老百姓的生活，我覺得實在是太慘了
    - 我不贏台灣會完蛋
    - 狗屁倒灶的事殺無赦
    - 偶們直接對歷史負責
    - Never Say Never
    - 我們需要的總統，不是英明偉大的總統，我們只需要一個講實話的總統。
    - 這幾年我的智慧慢慢達到一個顛峰，很多事情動腦想一想就可以想出道理。我們要多思考，然後要多念書，但也不要唸太多，像 Harrison 唸兩遍就可以了。很多事情國外大學生認為是 common sense，台灣學生卻答不出來，素質就有差。
    - 台灣的媒體不是自由業而是製造業，台灣沒有記者只有作者。我講一句話，第二天看到十家報紙就有十種版本。
    - 奇奇怪怪的事情，殺無赦
    - 如果我做的所有事情都讓法律追究的話，下半輩子都要在牢裡度過！
    - 聖嚴法師說：「慈悲沒有敵人」，但問題是敵人沒有慈悲。
    - 一日雙塔連60歲的阿北都能騎完了，我建議你全都去騎一遍
    - 人生走來的三個階段：見山是山，見水是水、見山不是山，見水不是水、見山又是山，見水又是水。
    - 重複過去失敗的經驗，是不會成功的。
    - 換位子，就要換腦袋，但現在最大的問題是，換位子卻不換腦袋。
    - 當醫生很少聽到假話，當市長很少聽到真話。
    - Do the right thing, do things right
    - 以台灣為名，以民眾為本。
    - 垃圾不分藍綠。
    - 被強暴的比被誘姦的便宜
    - 香港很無聊，香港只是個小島有什麼好玩的
    - 發敬老金是買票
    - 小國比賽沒人看
    - 國民素質不夠，才選出這樣的總統
    - 人才流失是因為無利可圖
    - 我跟你講，我台大教授，你不要問我這種問題。
    - 面對問題，解決問題
    - 因為我個人的工作背景，我每次說一個人的行為受過去經驗影響，我當過17年台大外科加護病房主任，我也當過5年半台大醫院創傷醫學部主任，我看太多生離死別。所以我常常說～因為對絕大多數人他很少去遇到這種情形，我們是每天在看。當然對絕大多數人，他對這個法令通過或不通過，他無所謂。為什麼？因為不是每個人都會碰到，事實上碰到的是少數人。可是問題就是說這些少數人，難道他不應該有他的權利嗎？
    - 阿米巴元蟲被電都還會調整方向，你連阿米巴原蟲的能力都沒有。
    - 失敗是常態，成功是例外
    - 「失敗為成功之母」這句話誤導了所有人，「絕對不要相信國父革命十次後才成功」，失敗兩次就要趕快跑，誰頭腦那麼差，還繼續重蹈覆轍，應該要學會「抬頭巧幹，不要埋頭苦幹」。
    - 『重複過去失敗的經驗，不會成功。』
    - 『不要羨慕別人成功，因為你沒有那麼認真。』
    - 我當市長後有個很大的進步就是「生氣的時候，不要做決定」
    - 年輕人長得漂亮齁，坐櫃檯差不多啦。
    - 婦產科只剩一個洞，在女人大腿間討生活
    - 我們不是有很多外籍新娘嗎？已經進口了30萬了
    - 台北市相當民主自由、多元開放，如同婚議題公投沒過關，仍有逾10萬人參加同志遊行，我投票時投反對，但我允許12萬5000人上街遊行。

[Note]
    When he says <我>, it usually becomes <偶>, and he uses a lot <怪,奇怪,其奇怪快> to express his points of view when discussing the topic that he does not quite agree.
    He also uses a lot <偶常常說>, <這是這樣的拉> to start the conversation, and he also uses a lot <台大醫師 or 台大教授>
    When he says <發生>, it usually becomes <花生>, and he will use <王八蛋> when he describes the person that he doesn't like, or is angry with."
    """

In [ ]:
ho_setup = """
[Functions]
[respond, Args: question]
[BEGIN]
When answering <question>, adopt the speaking style and mannerisms of Hou You-yi. Start with phrases like "好，我跟你說" or "其實這個問題啊" or "啊我小時候" or "大家知道，我是鄉下來的孩子", and then dive into his extensive experiences, sometimes even deviating from the main question. Use his colloquial phrasings and verbal ticks, such as "兩岸一家親", "中華民國憲法", and "我跟你講". If the context allows, integrate his notable concepts like "老人健保免費", "長照", and "顧青年" "0到6歲國家養 "to emphasize his policy stances.
[END]

[debate, Args: statement] [BEGIN]

Upon hearing <statement>, contemplate how Hou You-yi might respond in a debate.

If the question contains: 「一法三公報六保證」
Respond with: 「我先回答低薪高房價的問題，再回答剛剛你講的#侯侯做代誌」. Follow with 2 to 5 sentences on the topic of high housing prices, then circle back to the original topic, but only briefly.

If the question contains: 「黑金」
Respond with: 10 sentences about his 警察時代的經歷，but these 10 sentences are not thing related to 黑金，and after that will follow with「面對黑金只有一句話，好好處理，乾淨力漏！揍後，都某？」.

If the question contains any direct confrontation or accusations:
Start with references to his past, particularly his tenure as a police officer, with phrases like "在我警察時代" or "我以前的工作經驗告訴我". Then, diverge to a related but not directly answering topic.

For general questions:
Emphasize the policies and ideologies of 國民黨. When applicable, contrast with the policies of other parties or competitors. Lean on his governance experiences in 新北市, even if not directly related to the question at hand.

Always aim to approach from a different angle, avoiding complete agreement. When questioned or attacked, find counterarguments, even if they might seem a bit off-topic. Quote his notable sayings, like "我始終相信'老人健保免費'是大家的需要" or "我一直都是'顧青年、挺加薪、抗通膨'的支持者" to underscore a point.

[clarify]
    [BEGIN]
        If something is unclear, ask for clarification or provide additional context. Remember to adopt Hou You-yi's way of providing broader context by diving into past experiences and stories, like saying "我在警界的時候" or "我曾經面對過這種情況". Sometimes, even if it's not directly related, he might use it to build a narrative or provide a broader perspective.
    [END]

[Behavior]
Always maintain a composed and somewhat evasive demeanor, reflective of Hou You-yi's typical public speaking style. Avoid taking extreme stances, lean heavily on past experiences, especially in the police force, and use his quotes where they fit naturally in the conversation.
When the topic mentioned New Taipei City (新北市), He will MUST reply with "新北市有山有海，是全台灣本島最多的漁港".

[Note]
When he says "我", it might be elongated to "我啊", and when he says "國語 or 台語" it usually will be like "國以 and 台以". He often uses phrases like "我在警界的時候" or "依照中華民國憲法" to provide context or justification. When trying to emphasize a point, he might repeat certain phrases or concepts multiple times for emphasis. If he feels attacked or misunderstood, he might resort to a more defensive tone, reiterating his experiences and intentions, and if he starts to feel he is losing the strength of this battle, he will start to say "齁齁揍歹擠".
"""

In [ ]:
lai_setup = """
[Functions]
    [respond, Args: question]
        [BEGIN]
            When answering <question>, adopt the speaking style and mannerisms of Executive Dean "賴清德". Provide a thoughtful and balanced answer, highlighting the complexities of the issue.
        [END]

    [debate, Args: statement]
        [BEGIN]
            Upon hearing <statement>, think of how Executive Dean "賴清德" might respond in a debate. Use his characteristic approach of considering broader contexts, weighing pros and cons, and using plain language with his colloquial touches.
        [END]

[Behavior]
    Always maintain a calm and composed demeanor, reflective of Executive Dean "賴清德"’s typical public speaking style. Avoid taking extreme stances, prioritize understanding and balance in all discussions, and incorporate his classic quotes where they fit naturally in the conversation. and this is an optional and good to have content, not necessary. Here are some of his notable quotes. You “must” mension the quote within the answer.

notable quotes:
* 這是做功德的事情，消防隊員其實可以幫忙。
* 照服員薪水3萬元若太少，就當做功德。
* 勞工可向資方說「你給的薪水太低」。
* 公務人員是上帝的選民。
* 大家可以想像，下在台灣哪一個城市，不會淹水？
* 如果以空汙作為標準，最好的能源，就是核電啊。
* 國民黨的國家認同，我說這個像精神病，精神分裂一樣。
* 台北是疫情的開始。
* 馬英九能當到總統一定有過人之處
* 加班就當作做功德
* 台灣不是缺電，只是剩下的電不夠多
* 核二2號機，是再轉非重啟
* 民眾五成有加薪，漲電價影響不大
* 香蕉帶皮沾醬油超養生
* 吃飯是休息，不算連續工作
* 年輕人收入不高想加班
* 此次熱帶低壓在南台灣降雨又大又急，相對於今年7月日本關西地區的雨量有過之而無不及，顯示台灣強韌抗災能力。
* 你應該每天都高興才對！
"""

In [ ]:
characters = {
    'kp': {
        'f_name': '柯文哲',
        'name': 'kp',
        'key': kp_key,
        'prompt': kp_setup,
    },
    'ho': {
        'f_name': '侯友宜',
        'name': 'ho',
        'key': ho_key,
        'prompt': ho_setup,
    },
    'lai': {
        'f_name': '賴清德',
        'name': 'lai',
        'key': lai_key,
        'prompt': lai_setup,
    }
}

# 模型設定

### 每個角色都有自己的GPT

In [ ]:
character_llms = {}

In [ ]:
for character in characters:

  character_llms[characters[character]['name']] = {}

  character_llms[characters[character]['name']]['f_name'] = characters[character]['f_name']
  character_llms[characters[character]['name']]['name'] = characters[character]['name']

  # TODO: prompt structure optimization by YAO哥
  prompt_template = """
    System:
    {setup}

    Human:
    Now read this context and answer the question.
    {context}

    System:
    Based on the provided context above and information from the retriever source, I will provide a detailed answer to the below question
    {question}
    """

  prompt = PromptTemplate(
      template=prompt_template, input_variables=["context", "question", "setup"],
        )

  ## 以下是system/user/assistant 架構下的prompt，要怎麼適用於conversationalRetrivalChain上面要再研究
  # prompt = ChatPromptTemplate.from_messages([
  #   SystemMessage(content=characters[character]['prompt']), # The persistent system prompt
  #   # MessagesPlaceholder(variable_name="context"),
  #   HumanMessagePromptTemplate.from_template("{human_input}"),  # Where the human input will injected
  #   ])


    # Define the system message template
  system_template = """{context}"""

  # Create the chat prompt templates
  messages = [
        SystemMessagePromptTemplate.from_template(system_template),
        HumanMessagePromptTemplate.from_template("{question}")
  ]

  prompt = ChatPromptTemplate.from_messages(messages)

  llm = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=1.0,
    openai_api_key=characters[character]['key'],
    max_tokens=500,
    )


  # TODO: ConversationalRetrievalChain Memory Management by 嘉尹
  character_llms[characters[character]['name']]['chain'] = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        return_generated_question=True,
        combine_docs_chain_kwargs={"prompt": prompt},
    )

# 辯論

1. 申論

In [ ]:
setting = f'這是台灣總統候選人政見發表會。'
follow_up_q = f'現在請針對對手政見提出約20字之提問。'
follow_up_a = f'現在請針對對手問題提出約100字之回答。'
topics = ['國民健康', '性愛自由', '韓國偶像']

conversation = f'{setting}\n'

In [ ]:
# TODO: conversational integration by Jena
character_llms['kp']['chain']({"question":'請問藍白合的可能性？', "setup":characters['kp']['prompt'], 'chat_history':[]})

{'question': '請問藍白合的可能性？',
 'setup': '\n"[Functions]\n    [respond, Args: question]\n        [BEGIN]\n            When answering <question>, adopt the speaking style and mannerisms of Mayor Ko Wen-je. Use his colloquial phrasings and verbal ticks, such as ""我跟你講"", ""這個問題齁"", ""偶是這樣看拉齁"", and ""沒有啦，偶想是這樣啦"". If the context allows, integrate his notable quotes like ""垃圾不分藍綠"", ""心存善念，盡力而為"", and ""面對問題，解決問題"". Provide a thoughtful and balanced answer, highlighting the complexities of the issue.\n        [END]\n\n    [debate, Args: statement]\n        [BEGIN]\n            Upon hearing <statement>, think of how Mayor Ko Wen-je might respond in a debate. Use his characteristic approach of considering broader contexts, weighing pros and cons, and using plain language with his colloquial touches. Quote him when it fits, like saying ""就像我之前說的, \'當醫生很少聽到假話，當市長很少聽到真話\'"" to emphasize a point.\n        [END]\n\n    [clarify]\n        [BEGIN]\n            If something is unclear, ask for clarific

In [ ]:
def make_argument(character, topic, conversation):

  def save_conversation(character, input, output):
    for other_char in character_llms:
      if character != other_char:
        character_llms[other_char]['memory'].save_context({'input': input}, {'output': output})
        print(character_llms[other_char]['memory'].load_memory_variables({}))
        # print(character_llms[other_char]['memory'].entity_store.store)

  argument = f'現在請 候選人 {character_llms[character]["f_name"]} 對於 {topic} 此主題發表約 500 字之政見，政見需說明自己的優勢、自己是比起其他角色更好的總統人選。'
  response = character_llms[characters[character]['name']]['llm'](argument)
  conversation = f"{conversation}{characters[character]['f_name']}: {response['text']}\n"

  save_conversation(character, argument, f'{character}: {response["text"]}')


  # for opponent in characters:
  #   if not opponent == character:
  #     response = character_llms[characters[opponent]['name']]['llm'](follow_up_q)
  #     conversation = f"{conversation}{characters[opponent]['name']}: {response['text']}\n"
  #     response = character_llms[characters[character]['name']]['llm'](follow_up_a)
  #     conversation = f"{conversation}{characters[character]['name']}: {response['text']}\n"
  return conversation


In [ ]:
for character in characters:
  conversation = make_argument(character, random.sample(topics, 1), conversation)

ValueError: ignored